# Melhor modelo LSTM padrao ate o momento R2 de .71

In [125]:
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sympy import false
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer, KNNImputer
from copy import deepcopy as dc
import logging
from datetime import datetime
import optuna
import torch.nn.functional as F
import os

In [126]:
# Configuração inicial
data_hoje = datetime.now().strftime('%d-%m')
inicio_execucao = pd.Timestamp.now()

# Criando diretórios para logs e plots
os.makedirs(f'../logs/{data_hoje}', exist_ok=True)
os.makedirs(f'../plots/{data_hoje}', exist_ok=True)

# Configuração do logging
logging.basicConfig(filename=f'../logs/{data_hoje}/lstm_optuna.log', level=logging.INFO, format='- %(message)s')
logging.info('-' * 50)
logging.info(f'{inicio_execucao} - Iniciando o processo de otimização e treinamento do modelo LSTM')

# Carregando e preparando os dados
df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono'], low_memory=False)

df_original['Data e Hora'] = pd.to_datetime(df_original['Data e Hora'])
df_original.set_index('Data e Hora', inplace=True)
df_original.sort_index(inplace=True)

colunas_selecionadas = ['PM2.5', 'PM10', 'Monóxido de Carbono']
logging.info(f"Colunas selecionadas: {colunas_selecionadas}")
df = df_original[colunas_selecionadas]
df = df.loc['2019-01-01':'2022-01-01']

df = df.apply(pd.to_numeric, errors='coerce')

In [127]:

def treat_outliers(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
        df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df


# fazendo o logging de qual algoritmo de imputação foi utilizado
def log_imputation(method_name, impute_function, df):
    df_imputed = impute_function(df)
    logging.info(f"Imputação realizada usando: {method_name}")
    return df_imputed


# Função para imputação de dados ausentes
def backward_fill_imputer(df):
    df_imputed = df.fillna(method='bfill')
    return df_imputed


#mean imputation using the 24h mean
def mean_imputation(df):
    df_imputed = df.fillna(df.rolling(window=24, min_periods=1).mean())
    return df_imputed


def linear_interpolation_imputer(df):
    df_imputed = df.interpolate(method='linear')
    return df_imputed


# df_imputed = log_imputation('Backward Fill', backward_fill_imputer, df)
df_imputed = log_imputation('Linear Interpolation', linear_interpolation_imputer, df)
# df_imputed = log_imputation('Mean Imputation 24h', mean_imputation, df)

logging.info(f"Dados ausentes antes da imputação: {df.isna().sum()}")
logging.info(f"Dados ausentes após a imputação: {df_imputed.isna().sum()}")

# treating outliers
# df = treat_outliers(df)

In [128]:
# Preparando os dados para LSTM
def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)
    for col in colunas_selecionadas:
        for i in range(1, n_steps + 1):
            df[f'{col}(t-{i})'] = df[col].shift(i)
    df.dropna(inplace=True)
    return df


lookback = 8
shifted_df = prepare_dataframe_for_lstm(df_imputed, lookback)

# Dividindo em conjuntos de treino, validação e teste
train_size = int(len(shifted_df) * 0.7)
val_size = int(len(shifted_df) * 0.15)

train_df = shifted_df.iloc[:train_size]
val_df = shifted_df.iloc[train_size:train_size + val_size]
test_df = shifted_df.iloc[train_size + val_size:]

# Normalizando os dados de forma correta
scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = pd.DataFrame(scaler.fit_transform(train_df), columns=shifted_df.columns, index=train_df.index)
val_scaled = pd.DataFrame(scaler.transform(val_df), columns=shifted_df.columns, index=val_df.index)
test_scaled = pd.DataFrame(scaler.transform(test_df), columns=shifted_df.columns, index=test_df.index)

X_train, y_train = train_scaled.iloc[:, len(colunas_selecionadas):].values, train_scaled.iloc[:, 0].values
X_val, y_val = val_scaled.iloc[:, len(colunas_selecionadas):].values, val_scaled.iloc[:, 0].values
X_test, y_test = test_scaled.iloc[:, len(colunas_selecionadas):].values, test_scaled.iloc[:, 0].values

# Reshape para LSTM
X_train = X_train.reshape((-1, lookback, len(colunas_selecionadas)))
X_val = X_val.reshape((-1, lookback, len(colunas_selecionadas)))
X_test = X_test.reshape((-1, lookback, len(colunas_selecionadas)))
y_train = y_train.reshape((-1, 1))
y_val = y_val.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

# Convertendo para tensores PyTorch
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()


# Dataset e DataLoader
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]


# Modelo LSTM
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [129]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_dim = hidden_size // num_heads

        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        q = self.query(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.key(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.value(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attention_weights = F.softmax(scores, dim=-1)
        context = torch.matmul(attention_weights, v)

        context = context.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_size)
        output = self.out(context)
        return output

class ImprovedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_heads, dropout):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.attention = MultiHeadAttention(hidden_size * 2, num_heads)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        # LSTM layers
        lstm_out, _ = self.lstm(x)
        
        # Apply attention
        attn_out = self.attention(lstm_out)
        
        # Add & Norm
        out = self.layer_norm(lstm_out + attn_out)
        out = self.dropout(out)
        
        # Global average pooling
        out = torch.mean(out, dim=1)
        
        # Final prediction
        out = self.fc(out)
        return out


In [130]:
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import ReduceLROnPlateau


# Função de treinamento
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device):
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    best_val_loss = float('inf')
    epochs_without_improvement = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Treinamento
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output, y_batch)
            loss.backward()

            # Gradient clipping
            clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validação
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                x_batch, y_batch = batch[0].to(device), batch[1].to(device)
                output = model(x_batch)
                val_loss += criterion(output, y_batch).item()
        val_loss /= len(val_loader)

        # Ajuste da taxa de aprendizagem
        scheduler.step(val_loss)

        # logging.info(f"Época {epoch + 1}/{num_epochs}, Perda de treino: {train_loss:.4f}, Perda de validação: {val_loss:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            best_model_state = model.state_dict()
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            logging.info(f"Early stopping ativado na época {epoch + 1}")
            break

    # Carregar o melhor modelo
    model.load_state_dict(best_model_state)
    return model

In [131]:
def objective(trial):
    hidden_size = trial.suggest_int('hidden_size', 32, 256)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    num_heads = trial.suggest_int('num_heads', 1, 8)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)

    train_dataset = TimeSeriesDataset(X_train, y_train)
    val_dataset = TimeSeriesDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    input_size = X_train.shape[2]
    model = ImprovedLSTM(input_size, hidden_size, num_layers, num_heads, dropout).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = nn.HuberLoss()

    num_epochs = 100
    early_stopping_patience = 10

    model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, early_stopping_patience, device)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            output = model(x_batch)
            val_loss += criterion(output, y_batch).item()
    val_loss /= len(val_loader)

    return val_loss

In [132]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=20)  # Reduced number of trials for quicker results

best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

# Train the final model with the best hyperparameters
best_hidden_sizes = [best_params[f'hidden_size_{i}'] for i in range(best_params['num_layers'])]
best_batch_size = best_params['batch_size']

train_dataset = TimeSeriesDataset(X_train, y_train)
val_dataset = TimeSeriesDataset(X_val, y_val)
test_dataset = TimeSeriesDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)

final_model = ImprovedLSTM(X_train.shape[2], best_hidden_sizes, best_params['num_heads'], best_params['dropout']).to(device)
optimizer = torch.optim.AdamW(final_model.parameters(), lr=best_params['learning_rate'], weight_decay=best_params['weight_decay'])
criterion = nn.HuberLoss()

num_epochs = 100
patience = 10

final_model = train_model(final_model, train_loader, val_loader, criterion, optimizer, num_epochs, patience, device)

# Salvar o modelo final
torch.save(final_model.state_dict(), f'../models/best_model_optuna_{data_hoje}.pth')


# Avaliação final
def evaluate(model, dataloader):
    predictions = []
    actual = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            x_batch, y_batch = batch[0].to(device), batch[1].to(device)
            output = model(x_batch)
            predictions.extend(output.cpu().numpy().flatten())
            actual.extend(y_batch.cpu().numpy().flatten())
    return np.array(predictions), np.array(actual)


train_predictions, train_actual = evaluate(final_model, train_loader)
val_predictions, val_actual = evaluate(final_model, val_loader)
test_predictions, test_actual = evaluate(final_model, test_loader)

[I 2024-09-01 18:27:11,639] A new study created in memory with name: no-name-fa00a062-6655-4d56-9c15-377105823304
C:\dev\scripts\pm25-plots\venv\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
[I 2024-09-01 18:30:14,400] Trial 0 finished with value: 0.0007350033908639284 and parameters: {'hidden_size': 68, 'num_layers': 3, 'num_heads': 4, 'batch_size': 128, 'learning_rate': 0.003090445301104967, 'dropout': 0.48371815223298087, 'weight_decay': 3.945614028346779e-05}. Best is trial 0 with value: 0.0007350033908639284.
[W 2024-09-01 18:30:14,416] Trial 1 failed with parameters: {'hidden_size': 245, 'num_layers': 1, 'num_heads': 6, 'batch_size': 32, 'learning_rate': 0.0014616317229014918, 'dropout': 0.3509789337039221, 'weight_decay': 6.189184384966231e-05} because of the following error: RuntimeError("shape '[32, 8, 6, 81]' is invalid for input of size 125440").
Trace

RuntimeError: shape '[32, 8, 6, 81]' is invalid for input of size 125440

In [ ]:
# Desnormalização
def inverse_transform_data(normalized_data):
    # Criar um array de zeros com o mesmo número de colunas que os dados originais
    dummy_array = np.zeros((len(normalized_data), len(shifted_df.columns)))

    # Colocar os dados normalizados na primeira coluna (assumindo que é PM2.5)
    dummy_array[:, 0] = normalized_data

    # Aplicar a transformação inversa
    denormalized_data = scaler.inverse_transform(dummy_array)

    # Retornar apenas a primeira coluna, que contém os dados desnormalizados de interesse
    return denormalized_data[:, 0]


# Desnormalização das previsões e valores reais
train_predictions = inverse_transform_data(train_predictions)
val_predictions = inverse_transform_data(val_predictions)
test_predictions = inverse_transform_data(test_predictions)

# Para os valores reais, precisamos garantir que estamos usando os dados originais não normalizados
train_actual = df_imputed['PM2.5'].values[:len(train_predictions)]
val_actual = df_imputed['PM2.5'].values[len(train_predictions):len(train_predictions) + len(val_predictions)]
test_actual = df_imputed['PM2.5'].values[-len(test_predictions):]


# Cálculo das métricas
def calculate_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return rmse, mse, mae, r2, mape


# Calcular métricas
train_rmse, train_mse, train_mae, train_r2, train_mape = calculate_metrics(train_actual, train_predictions)
val_rmse, val_mse, val_mae, val_r2, val_mape = calculate_metrics(val_actual, val_predictions)
test_rmse, test_mse, test_mae, test_r2, test_mape = calculate_metrics(test_actual, test_predictions)

# Logging dos resultados
logging.info(
    f"Métricas de Treino: RMSE={train_rmse:.4f}, MSE={train_mse:.4f}, MAE={train_mae:.4f}, R2={train_r2:.4f}, MAPE={train_mape:.4f}")
logging.info(
    f"Métricas de Validação: RMSE={val_rmse:.4f}, MSE={val_mse:.4f}, MAE={val_mae:.4f}, R2={val_r2:.4f}, MAPE={val_mape:.4f}")
logging.info(
    f"Métricas de Teste: RMSE={test_rmse:.4f}, MSE={test_mse:.4f}, MAE={test_mae:.4f}, R2={test_r2:.4f}, MAPE={test_mape:.4f}")

In [ ]:

# Log das métricas finais
logging.info("\nMétricas finais:")
logging.info("Treinamento - RMSE: {:.4f}, MSE: {:.4f}, MAE: {:.4f}, R²: {:.4f}".format(train_rmse, train_mse, train_mae,
                                                                                       train_r2))
logging.info(
    "Validação - RMSE: {:.4f}, MSE: {:.4f}, MAE: {:.4f}, R²: {:.4f}".format(val_rmse, val_mse, val_mae, val_r2))
logging.info(
    "Teste - RMSE: {:.4f}, MSE: {:.4f}, MAE: {:.4f}, R²: {:.4f}".format(test_rmse, test_mse, test_mae, test_r2))

# Plotagem dos resultados
plt.figure(figsize=(12, 6))
plt.plot(train_actual, label='Actual PM2.5')
plt.plot(train_predictions, label='Predicted PM2.5')
plt.title('Treinamento: PM2.5 Real vs Previsto')
plt.xlabel('Hora')
plt.ylabel('PM2.5')
plt.legend()
plt.savefig(f'../plots/{data_hoje}/lstm_optuna_train_{data_hoje}.png')


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

train_dates = shifted_df.index[:len(train_actual)]
val_dates = shifted_df.index[len(train_actual):len(train_actual) + len(val_actual)]
test_dates = shifted_df.index[-len(test_actual):]


def plot_results(actual, predicted, dates, title, filename):
    plt.figure(figsize=(20, 12))
    plt.plot(dates, actual, label='Real', color='blue')
    plt.plot(dates, predicted, label='Previsto', color='red')
    plt.title(title)
    plt.xlabel('Data')
    plt.ylabel('PM2.5')
    plt.legend()

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Mostrar a cada 3 meses

    plt.gcf().autofmt_xdate()  # Rotacionar e alinhar os rótulos de data
    plt.tight_layout()
    plt.savefig(f'../plots/{data_hoje}/{filename}_{data_hoje}.png')
    plt.close()


plot_results(train_actual, train_predictions, train_dates, 'Treinamento: PM2.5 Real vs Previsto', 'lstm_optuna_train')
plot_results(val_actual, val_predictions, val_dates, 'Validação: PM2.5 Real vs Previsto', 'lstm_optuna_val')
plot_results(test_actual, test_predictions, test_dates, 'Teste: PM2.5 Real vs Previsto', 'lstm_optuna_test')

fim_execucao = pd.Timestamp.now()
tempo_execucao = fim_execucao - inicio_execucao
logging.info(f"\nExecução finalizada em {fim_execucao}")
logging.info(f"Tempo total de execução: {tempo_execucao}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd


def plot_results_by_month(actual, predicted, dates, title_prefix, filename_prefix):
    df = pd.DataFrame({'date': dates, 'actual': actual, 'predicted': predicted})
    df.set_index('date', inplace=True)

    grouped = df.groupby(pd.Grouper(freq='M'))

    for name, group in grouped:
        if len(group) > 0:
            plt.figure(figsize=(12, 6))
            plt.plot(group.index, group['actual'], label='Real', color='blue')
            plt.plot(group.index, group['predicted'], label='Previsto', color='red')

            month_year = name.strftime('%B %Y')
            plt.title(f'{title_prefix} - {month_year}')
            plt.xlabel('Data')
            plt.ylabel('PM2.5')
            plt.legend()

            plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m'))
            plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=5))

            plt.gcf().autofmt_xdate()  # Rotacionar e alinhar os rótulos de data
            plt.tight_layout()

            month_filename = f'{filename_prefix}_{name.strftime("%Y_%m")}_{data_hoje}.png'
            plt.savefig(f'../plots/{data_hoje}/{month_filename}')
            plt.close()


plot_results_by_month(train_actual, train_predictions, train_dates, 'Treinamento: PM2.5 Real vs Previsto',
                      'lstm_optuna_train')
plot_results_by_month(val_actual, val_predictions, val_dates, 'Validação: PM2.5 Real vs Previsto', 'lstm_optuna_val')
plot_results_by_month(test_actual, test_predictions, test_dates, 'Teste: PM2.5 Real vs Previsto', 'lstm_optuna_test')